## 🥁 Setup

Change the following variable settings to match your deployed model's *Inference endpoint*. for example: 

```
deployed_model_name = "baklava"
infer_endpoint = "https://baklava-yyyyyy.apps.cluster-p9k5m.p9k5m.sandboxxxx.opentlc.com"
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import joblib

In [ ]:
deployed_model_name = "baklava"
infer_endpoint = "<YOUR_INFERENCE_ENDPOINT>"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

In [ ]:
# the version you chose to test
version = "<YOUR_VERSION>"

In [ ]:
# Load the scaler (assuming it's saved as scaler.pkl)
scaler = joblib.load(f"models/{version}/baklava/1/artifacts/scaler.pkl")

# Define your data 
data = {
  "Holiday_Promotion": 1, 
  "Population_per_Bakery": 10000, 
  "Income_Level": 20000, 
  "Holiday_Season": 1,
  "Region_Aegean": 0,  
  "Region_Black Sea": 1,
  "Region_Central Anatolia": 0,
  "Region_Eastern Anatolia": 0,
  "Region_Marmara": 0,
  "Region_Mediterranean": 0,
  "Region_Southeast Anatolia": 0
}

# Convert data to a pandas DataFrame (assuming your features are column names)
df_input = pd.DataFrame([data])

# Reorder columns to match the order used during training (if necessary)
df_input = df_input[scaler.feature_names_in_] 

# Scale the data
scaled_data = scaler.transform(df_input).astype(np.float32)
scaled_data = scaled_data.reshape(1, -1)  

# Define the request data
json_data = {
  "inputs": [
    {
      "name": "input",
      "shape": [1, len(scaled_data[0])],  
      "datatype": "FP32",
      "data": scaled_data.tolist()[0] 
    }
  ]
}

# Send the request
response = requests.post(infer_url, json=json_data, verify=True)

# Process the response (assuming similar to the example)
if response.status_code == 200:
  response_dict = response.json()
  predicted_demand = response_dict['outputs'][0]['data']
  print(f"Predicted Baklava Consumption: {predicted_demand}")
else:
  print(f"Error: {response.status_code}")